In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim
from gensim.models import Word2Vec
import gensim.downloader as api

In [5]:
# Print all available models in gensim-data
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [6]:
# Download the model and return as object ready for use
model = api.load("word2vec-google-news-300")

2023-06-12 21:05:53,864 : INFO : loading projection weights from C:\Users\laure/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2023-06-12 21:06:49,731 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\laure/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-06-12T21:06:49.716990', 'gensim': '4.3.1', 'python': '3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}


In [7]:
# Read json file
import json
with open('../data/recipe_scrapped_data.json', encoding="utf8") as f:
    data = json.load(f)


#print(data[0])
# Get the first list of ingredients
ingredients = data[0]['ingredient']

# Remove brackets
ingredients = ingredients.replace('[', '')
ingredients = ingredients.replace(']', '')

# Print the list of ingredients
print(ingredients)

# Get the first ingredient
ingredient = ingredients.split(',')[0]

# Print the ingredient
print(ingredient)



"d'oignon émincés", "reblochon bien fait (lorsqu'on appuie sur le côté du reblochon, le doigt doit s'enfoncer un peu)", "d'huile", 'sel', 'poivre', 'de pomme de terre', 'de lardons fumés', "d'ail"
"d'oignon émincés"


In [8]:
# With my list of ingredients, and the model, give me the most similar ingredients
pairs = [ ("vin", "bière") ]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model.similarity(w1, w2)))

'vin'	'bière'	0.50


In [9]:
from utils import *

In [12]:
# I use a memory friendly iterator
text_w2v = MySentences(dirname='./Data/')


model_w2v = gensim.models.Word2Vec(sentences=text_w2v)
if not os.path.exists('./WE_models'):
    os.mkdir('./WE_models')
model_w2v.save('WE_models/w2v_cbow_300D')

2023-06-12 21:07:57,070 : INFO : collecting all words and their counts
2023-06-12 21:07:57,091 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-12 21:07:57,128 : INFO : PROGRESS: at sentence #10000, processed 74155 words, keeping 7544 word types
2023-06-12 21:07:57,164 : INFO : PROGRESS: at sentence #20000, processed 147626 words, keeping 11105 word types
2023-06-12 21:07:57,202 : INFO : PROGRESS: at sentence #30000, processed 221497 words, keeping 13820 word types
2023-06-12 21:07:57,244 : INFO : PROGRESS: at sentence #40000, processed 297254 words, keeping 16006 word types
2023-06-12 21:07:57,276 : INFO : PROGRESS: at sentence #50000, processed 372445 words, keeping 18020 word types
2023-06-12 21:07:57,311 : INFO : PROGRESS: at sentence #60000, processed 449189 words, keeping 19981 word types
2023-06-12 21:07:57,343 : INFO : PROGRESS: at sentence #70000, processed 529765 words, keeping 21707 word types
2023-06-12 21:07:57,380 : INFO : PROGRESS: at se

In [14]:
model_w2v = gensim.models.Word2Vec.load('WE_models/w2v_cbow_300D')

2023-06-12 21:08:48,621 : INFO : loading Word2Vec object from WE_models/w2v_cbow_300D
2023-06-12 21:08:48,655 : INFO : loading wv recursively from WE_models/w2v_cbow_300D.wv.* with mmap=None
2023-06-12 21:08:48,657 : INFO : setting ignored attribute cum_table to None
2023-06-12 21:08:48,909 : INFO : Word2Vec lifecycle event {'fname': 'WE_models/w2v_cbow_300D', 'datetime': '2023-06-12T21:08:48.909537', 'gensim': '4.3.1', 'python': '3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'loaded'}


In [15]:
list_ingredient = ['Filet de boeuf', 'Pomme de terre', 'Oignon', 'Carotte', 'Beurre', 'Huile d\'olive', 'Sel', 'Poivre']
list_wine = [['Fromage doux', 'Poisson maigre'], ["Viande rouge", "Boeuf"], ['Pain', 'Fromage', 'Soupe', 'Viande rouge']]

# Split the list of ingredients
list_ingredient = [ingredient.split(' ') for ingredient in list_ingredient]
# Concatenate the list of ingredients into a single list
list_ingredient = [item for sublist in list_ingredient for item in sublist]

# For each sublist in the list of wine, split the sublist
list_wine = [[ingredient.split(' ') for ingredient in sublist] for sublist in list_wine]
# For each sublist in the list of wine, concatenate the sublist into a single list
list_wine = [[item for sublist in ingredient for item in sublist] for ingredient in list_wine]

# All the word to lower case
list_ingredient = [ingredient.lower() for ingredient in list_ingredient]
list_wine = [[ingredient.lower() for ingredient in sublist] for sublist in list_wine]

# For the list of ingredients, generate the embedding list
list_ingredient_embedding = [model_w2v.wv[ingredient] for ingredient in list_ingredient]

# Average the embedding list to get a single vector
list_ingredient_embedding = np.mean(list_ingredient_embedding, axis=0)

print(list_ingredient_embedding)

# Same for the list of wine
list_wine_embedding = [[model_w2v.wv[ingredient] for ingredient in sublist] for sublist in list_wine]

list_wine_embedding = [np.mean(ingredient, axis=0) for ingredient in list_wine_embedding]

print(list_wine_embedding)

[-0.40819433  0.10852652 -0.27036893 -0.21170881  0.07396933 -0.18438849
 -0.03233916  0.21374895  0.00285567 -0.06408879  0.00924308 -0.44264394
  0.09184892  0.06409059  0.01744415 -0.01714859 -0.3931947  -0.10337509
  0.0644596  -0.21476582 -0.03659047  0.03381592  0.18158454  0.03064668
  0.10850547 -0.22574736 -0.12580241  0.09805565 -0.2579457  -0.45876142
 -0.06986421 -0.18832406  0.5381993  -0.21187383 -0.19750503  0.248252
 -0.4636482  -0.28617293 -0.20462674 -0.18679383 -0.17215648  0.07261482
 -0.11258549 -0.2522122   0.43186373 -0.19882001 -0.05292751 -0.2743212
 -0.27928466  0.56643873  0.18821877 -0.0944486   0.02578624  0.22338286
 -0.17111237  0.00457901  0.16115703  0.03436211  0.05333115 -0.5404745
  0.04222417  0.1359099  -0.06266882  0.23233058 -0.06509229  0.0915222
  0.19130021 -0.15015681 -0.22458312 -0.22595054 -0.45136422 -0.04332982
  0.25372064  0.01515401  0.38724756  0.35089985 -0.39490312  0.5997628
  0.24543437  0.11802909 -0.08668191 -0.29399532  0.08541

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity between the ingredient and the wine
list_similarity = [cosine_similarity(list_ingredient_embedding.reshape(1, -1), ingredient.reshape(1, -1)) for ingredient in list_wine_embedding]

print(list_similarity)

[array([[0.53145725]], dtype=float32), array([[0.6237839]], dtype=float32), array([[0.6408114]], dtype=float32)]


In [ ]:
#https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

# Word embedding

This notebooks contains the code to create word embeddings for recipes in order to link with the ingredients to a wine.

In [1]:
import json
import numpy as np
from deep_translator import GoogleTranslator

In [91]:
# Open the json file
#with open('../data/recipe_transformed_data.json', encoding="utf8") as f:
#    data_raw = json.load(f)

# Open the json file and take only the first 10 recipes
with open('../data/recipe_transformed_data.json', encoding="utf8") as f:
    data_raw = json.load(f)

data_raw['recipes'] = data_raw['recipes'][:10]

# Set the translator
translator = GoogleTranslator(source='auto', target='en')

test_data = data_raw['recipes'][0]
print(test_data)

FileNotFoundError: [Errno 2] No such file or directory: '../data/recipe_transformed_data.json'

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]    | 

True

In [4]:
custom_stopwords = set(stopwords.words('english') + ["sel"])

In [5]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [42]:
def generate_embeddings(ingredients):
    tokens = []
    for ingredient in ingredients:
        ingredient = translator.translate(ingredient)
        ingredient = ingredient.lower()
        ingredient_tokens = word_tokenize(ingredient)
        ingredient_tokens = [token for token in ingredient_tokens if token not in string.punctuation]
        ingredient_tokens = [token for token in ingredient_tokens if token not in custom_stopwords]
        tokens.extend([token for token in ingredient_tokens])
    tokens = list(set(tokens))
    tokens = [token.replace('game', 'wildfowl') for token in tokens]
    #print(tokens)
    return [model[token] for token in tokens if token in model]

# Testing np.mean

In [85]:
tab1 = [[2, 6 , 7], [4, 4, 5]]
tab2 = [[1, 5, 8], [3, 3, 6], [5, 2, 7]]
mean1 = np.mean(tab1, axis=0)
print(mean1)
mean2 = np.mean(tab2, axis=0)
print(mean2)
mean3 = np.mean([mean1] + [mean2]*2, axis=0)
print(mean3)
mean4 = np.mean(tab1 + tab2 * 2, axis=0)
print(mean4)

[3. 5. 6.]
[3.         3.33333333 7.        ]
[3.         3.88888889 6.66666667]
[3.   3.75 6.75]


In [87]:
error_counts = 0
data = data_raw['recipes'].copy()
for recipe in data:
    try:
        ingredient_embeddings = generate_embeddings(recipe['ingredients'])
        title_embeddings = generate_embeddings([recipe['name']])
        if len(ingredient_embeddings) == 0 and len(title_embeddings) == 0:
            # Delete the recipe
            data.remove(recipe)
            error_counts += 1
            continue
        elif len(ingredient_embeddings) == 0:
            recipe_embedding = np.mean(title_embeddings, axis=0)
        elif len(title_embeddings) == 0:
            recipe_embedding = np.mean(ingredient_embeddings, axis=0)
        else:
            mean_ingredient_embedding = np.mean(ingredient_embeddings, axis=0)
            #print(mean_ingredient_embedding)
            mean_title_embedding = np.mean(title_embeddings, axis=0)
            #print(mean_title_embedding)
            recipe_embedding = np.mean([mean_ingredient_embedding] + [mean_title_embedding]*2, axis=0)
            #print(mean_recipe_embedding)
            #recipe_embedding = np.mean(ingredient_embeddings + title_embeddings * 2, axis=0)     
            #print(recipe_embedding)
        recipe['pairings_embedding'] = recipe_embedding.tolist()
    except Exception as e:
        print("Error with recipe: {}".format(recipe['name']))
        print(e)
        # Delete the recipe
        data.remove(recipe)
        error_counts += 1
        continue

print("There were {} errors out of {} recipes".format(error_counts, len(data_raw['recipes'])))

There were 0 errors out of 1 recipes


In [88]:
print(data)

[{'name': 'Tartiflette : la vraie recette', 'type': 'Plat principal', 'ingredients': ['oignon émincés', 'reblochon bien fait', 'huile', 'sel', 'poivre', 'de pomme de terre', 'de lardons fumés', 'ail'], 'url': 'https://www.marmiton.org/recettes/recette_la-vraie-tartiflette_17634.aspx', 'pairings_embedding': [-0.08777472376823425, 0.026766689494252205, 0.027923583984375, 0.1764174848794937, -0.019481929019093513, 0.1298457384109497, 0.09827199578285217, -0.12731502950191498, -0.06656160950660706, 0.10516049712896347, -0.03183075785636902, -0.16708527505397797, -0.02421308122575283, 0.0057829259894788265, -0.17335957288742065, 0.2151162475347519, 0.07440185546875, 0.0579427070915699, -0.13278166949748993, -0.1676500290632248, 0.05337308347225189, 0.14039410650730133, 0.0776638463139534, -0.05036001279950142, 0.0037098124157637358, -0.020422732457518578, -0.10242068767547607, 0.07430260628461838, -0.03679202124476433, 0.10315064340829849, -0.12068746238946915, -0.07795114070177078, 0.05209

In [89]:
# Save the data
with open('../data/recipe_transformed_data.json', 'w', encoding="utf8") as f:
    json.dump(data, f)